In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import warnings
from scipy.stats import poisson
from tqdm import tqdm 
from scipy.stats import poisson
warnings.filterwarnings('ignore')


In [25]:
years = ['2018', '2019', '2020', '2021', '2022', '2023']
file_paths = [
    'data/VFT_Count_Zip_2018.csv',
    'data/VFT_Count_Zip_2019.csv',
    'data/VFT_Count_Zip_2020.csv',
    'data/VFT_Count_Zip_2021.csv',
    'data/VFT_Count_Zip_2022.csv',
    'data/VFT_Count_Zip_2023.csv'
]

# Load files with column renaming for VFT_Count_Zip_2023.csv
dfs = []
for file in file_paths:
    if '2023' in file:
        df = pd.read_csv(file).rename(columns={'ZIP Code': 'Zip Code'})
    else:
        df = pd.read_csv(file)
    dfs.append(df)

# Concatenate all data into one DataFrame
vft_data = pd.concat(dfs)

# Replacement dictionary
replacements = {
    '12/31/2023': '2023',
    '12/31/2022': '2022',
    '1/1/2022': '2021',
    '1/1/2021': '2020',
    '1/1/2020': '2019',
    '10/1/2018': '2018'
}

# Replace the values in the DataFrame
vft_data.replace(replacements, inplace=True)


In [31]:
stations_df = pd.read_csv('data/filtered_alt_fuel_stations.csv')

In [32]:

data = vft_data


In [33]:
merged_df = pd.merge(stations_df, vft_data, left_on='zip', right_on='Zip Code', how='inner')


In [34]:
merged_df

,access_code,access_days_time,access_detail_code,cards_accepted,date_last_confirmed,expected_date,fuel_type_code,groups_with_access_code,id,maximum_vehicle_class,...,federal_agency.id,federal_agency.code,federal_agency.name,Date,Zip Code,Model Year,Fuel,Make,Duty,Vehicles
0,public,24 hours daily,NaN,NaN,2024-08-15,NaN,ELEC,Public,6355,NaN,...,NaN,NaN,NaN,2018,92037,2006,Diesel and Diesel Hybrid,OTHER/UNK,Heavy,9
1,public,24 hours daily,NaN,NaN,2024-08-15,NaN,ELEC,Public,6355,NaN,...,NaN,NaN,NaN,2018,92037,2006,Diesel and Diesel Hybrid,OTHER/UNK,Light,15
2,public,24 hours daily,NaN,NaN,2024-08-15,NaN,ELEC,Public,6355,NaN,...,NaN,NaN,NaN,2018,92037,2006,Battery Electric,OTHER/UNK,Light,4
3,public,24 hours daily,NaN,NaN,2024-08-15,NaN,ELEC,Public,6355,NaN,...,NaN,NaN,NaN,2018,92037,2006,Flex-Fuel,OTHER/UNK,Light,13
4,public,24 hours daily,NaN,NaN,2024-08-15,NaN,ELEC,Public,6355,NaN,...,NaN,NaN,NaN,2018,92037,2006,Gasoline,ACURA,Light,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054196,public,24 hours daily,NaN,NaN,2024-12-03,NaN,ELEC,Public,353883,NaN,...,NaN,NaN,NaN,2022,91931,<2009,Unk,OTHER/UNK,Unk,14
3054197,public,24 hours daily,NaN,NaN,2024-12-03,NaN,ELEC,Public,353883,NaN,...,NaN,NaN,NaN,2022,91931,<2009,Diesel and Diesel Hybrid,OTHER/UNK,Heavy,4
3054198,public,24 hours daily,NaN,NaN,2024-12-03,NaN,ELEC,Public,353883,NaN,...,NaN,NaN,NaN,2022,91931,<2009,Diesel and Diesel Hybrid,OTHER/UNK,Light,12
3054199,public,24 hours daily,NaN,NaN,2024-12-03,NaN,ELEC,Public,353883,NaN,...,NaN,NaN,NaN,2022,91931,<2009,Flex-Fuel,OTHER/UNK,Light,11


In [35]:
merged_df = merged_df.dropna(axis=1, how='any')

In [36]:
data = merged_df

# Step 2: Ensure unique entries for each 'Zip Code' and 'Date' combination
aggregated_data = data.groupby(['Zip Code', 'Date'], as_index=False)['Vehicles'].sum()

# Step 3: Pivot the dataframe to have 'Zip Code' as rows and 'Date' as columns
pivot_data = aggregated_data.pivot(index='Zip Code', columns='Date', values='Vehicles')

# Step 4: Melt the dataframe back to long format
melted_data = pivot_data.reset_index().melt(
    id_vars=['Zip Code'], 
    var_name='date', 
    value_name='counts'
)

# Step 5: Fit a Poisson distribution for each 'Zip Code'
poisson_params = {}
for zip_code in melted_data['Zip Code'].unique():
    counts = melted_data[melted_data['Zip Code'] == zip_code]['counts'].dropna()
    lambda_param = np.mean(counts)  # Mean serves as the lambda parameter
    poisson_params[zip_code] = lambda_param

# Step 6: Convert Poisson parameters to a DataFrame for better visibility
poisson_params_df = pd.DataFrame.from_dict(poisson_params, orient='index', columns=['Lambda']).reset_index()
poisson_params_df.rename(columns={'index': 'Zip Code'}, inplace=True)

# Display the final Poisson parameters DataFrame
print(poisson_params_df)

# Optional: Save the results to a CSV file
output_path = 'result_data/poisson_parameters.csv' 
poisson_params_df.to_csv(output_path, index=False)


     Zip Code        Lambda
0       91901  4.696300e+04
1       91906  3.107400e+04
2       91910  1.387217e+06
3       91911  2.237094e+06
4       91913  3.995800e+04
..        ...           ...
104     92691  3.275893e+05
105     92692  4.336530e+05
106     92694  4.414980e+05
107     92697  9.170000e+02
108     95126  6.714750e+05

[109 rows x 2 columns]


In [37]:
pd.read_csv('poisson_parameters.csv')

,Zip Code,Lambda
0,91901,4.696300e+04
1,91906,3.107400e+04
2,91910,1.387217e+06
3,91911,2.237094e+06
4,91913,3.995800e+04
...,...,...
104,92691,3.275893e+05
105,92692,4.336530e+05
106,92694,4.414980e+05
107,92697,9.170000e+02


In [38]:


# Step 1: Load the data

data = merged_df

# Step 2: Ensure unique entries for each 'Zip Code' and 'Date' combination
aggregated_data = data.groupby(['Zip Code', 'Date'], as_index=False)['Vehicles'].sum()

# Step 3: Pivot the dataframe to have 'Zip Code' as rows and 'Date' as columns
pivot_data = aggregated_data.pivot(index='Zip Code', columns='Date', values='Vehicles')

# Step 4: Melt the dataframe back to long format
melted_data = pivot_data.reset_index().melt(
    id_vars=['Zip Code'], 
    var_name='date', 
    value_name='counts'
)

# Step 5: Fit a Poisson distribution for each 'Zip Code'
poisson_params = {}
for zip_code in melted_data['Zip Code'].unique():
    counts = melted_data[melted_data['Zip Code'] == zip_code]['counts'].dropna()
    lambda_param = np.mean(counts)  # Mean serves as the lambda parameter
    poisson_params[zip_code] = lambda_param

# Step 6: Monte Carlo Simulation
n_samples = 1000  # Number of simulations
samples_results = {}

for n in tqdm(range(n_samples), desc="Monte Carlo Simulation"):
    results = []
    for zip_code, lambda_param in poisson_params.items():
        # Generate a random sample from Poisson distribution
        value = poisson.rvs(mu=lambda_param, size=1)[0]
        results.append(value)
    samples_results[n] = results

# Convert simulation results to DataFrame
df_results = pd.DataFrame(samples_results).T  # Transpose so each sample is a column
df_results.columns = poisson_params.keys()  # Use Zip Codes as column names

# Step 7: Analyze results
df_results['row_sum'] = df_results.sum(axis=1)  # Total counts for each simulation
total_sum = df_results.sum(axis=0)  # Total counts for each Zip Code across simulations
row_means = df_results.mean(axis=1)  # Mean of row-wise totals

# Optional: Save results to a CSV
output_path = 'result_data/monte_carlo_results.csv'  # 替换为你的目标路径
df_results.to_csv(output_path, index=False)

# Print key results
print("Row-wise totals (sum):\n", df_results['row_sum'].head())
print("Column-wise totals (sum):\n", total_sum.head())
print("Row-wise means:\n", row_means.head())


Monte Carlo Simulation: 100%|██████████████| 1000/1000 [00:01<00:00, 615.34it/s]


Row-wise totals (sum):
 0    54342694
1    54333134
2    54344161
3    54326148
4    54342417
Name: row_sum, dtype: int64
Column-wise totals (sum):
 91901      46954572
91906      31079858
91910    1387211120
91911    2237144261
91913      39948700
dtype: int64
Row-wise means:
 0    988048.981818
1    987875.163636
2    988075.654545
3    987748.145455
4    988043.945455
dtype: float64


In [39]:
pd.read_csv('monte_carlo_results.csv')

,91901,91906,91910,91911,91913,91914,91915,91931,91932,91935,...,92675,92677,92679,92688,92691,92692,92694,92697,95126,row_sum
0,47055,31100,1387028,2235425,39819,86709,292872,633,110709,9248,...,254771,759438,27237,107446,326358,433118,441520,960,670578,54342694
1,46999,31058,1388287,2236734,39739,86594,292677,611,110827,9294,...,254305,758665,27110,107656,327284,433565,442683,967,671173,54333134
2,47059,30976,1387344,2235201,39823,86711,292809,615,110326,9277,...,255003,759545,27689,107528,328087,432745,441064,972,670770,54344161
3,47653,30902,1385668,2237064,39997,87055,292917,591,110736,9434,...,254990,758728,27923,107438,328443,434139,440062,924,673477,54326148
4,46692,31136,1387941,2238490,39649,87320,292924,615,110553,9239,...,254527,759147,27504,107147,328493,433430,440973,852,671089,54342417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,46667,31650,1386193,2238495,39620,87253,293236,606,110921,9387,...,255118,758318,27301,107594,326765,434441,441888,940,671238,54348713
996,47219,31037,1384582,2238614,40261,86528,292838,678,110509,9299,...,254892,758727,27600,107710,327231,433766,441091,883,672150,54326960
997,47240,31347,1387972,2235846,39900,86371,293200,566,110487,9287,...,254625,759813,27686,107724,327560,433855,442714,889,671868,54350518
998,46812,30997,1388270,2236737,39828,86709,294182,647,110685,9296,...,255603,758019,27513,107948,328763,432728,441676,925,671750,54348351


In [40]:
def analyze_ev_registrations(df):
    """
    Analyze EV registrations by zip code and fit Poisson distribution using mean rate
    
    Parameters:
    df (pandas.DataFrame): DataFrame with columns for zip code, year, and vehicle data
    
    Returns:
    pandas.DataFrame: Analysis results with zip codes and fitted lambda parameters
    """
    # Group by zip code and year to get counts
    ev_counts = df.groupby(['Zip Code', 'Date'])['Vehicles'].sum().reset_index()
    
    results = []
    
    # Calculate Poisson lambda (mean rate) for each zip code
    for zip_code in ev_counts['Zip Code'].unique():
        zip_data = ev_counts[ev_counts['Zip Code'] == zip_code]
        
        if len(zip_data) > 0:
            # For Poisson distribution, lambda is simply the mean rate
            lambda_param = zip_data['Vehicles'].mean()
            
            # Calculate goodness of fit using Chi-square test
            observed = zip_data['Vehicles'].values
            expected = np.full_like(observed, lambda_param)
            
            try:
                chi_square, p_value = stats.chisquare(observed, expected)
                
                results.append({
                    'zip': zip_code,
                    'lambda': lambda_param,
                    'total_evs': zip_data['Vehicles'].sum(),
                    'num_years': len(zip_data),
                    'avg_evs_per_year': lambda_param,
                    'p_value': p_value
                })
            except:
                results.append({
                    'zip': zip_code,
                    'lambda': lambda_param,
                    'total_evs': zip_data['Vehicles'].sum(),
                    'num_years': len(zip_data),
                    'avg_evs_per_year': lambda_param,
                    'p_value': None
                })
    
    return pd.DataFrame(results)

# Example usage:
results_df = analyze_ev_registrations(merged_df)


In [43]:
results_df.to_csv('result_data/result.csv', index=False)

In [46]:
def create_interactive_analysis(data_path, output_path='dashboard.html'):
    """
    Create interactive analysis and save as HTML
    
    Parameters:
    data_path: str - path to the CSV data file
    output_path: str - path where to save the HTML file
    """
    # Read the data
    df = pd.read_csv(data_path)
    
    # Create figures
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Top 10 ZIP Codes by Annual Average',
                       'Distribution of Registration Volumes',
                       'P-Value Distribution',
                       'Registration Volume vs P-Value'),
        specs=[[{"type": "bar"}, {"type": "histogram"}],
               [{"type": "histogram"}, {"type": "scatter"}]]
    )
    
    # 1. Top 10 ZIP codes bar chart
    top_10 = df.nlargest(10, 'lambda')
    fig.add_trace(
        go.Bar(
            x=top_10['zip'].astype(str),
            y=top_10['lambda'],
            name='Annual Average',
            text=top_10['lambda'].round(0),
            textposition='auto',
            hovertemplate="ZIP: %{x}<br>Annual Average: %{y:,.0f}<extra></extra>"
        ),
        row=1, col=1
    )
    
    # 2. Distribution of registration volumes
    fig.add_trace(
        go.Histogram(
            x=df['lambda'],
            name='Registration Distribution',
            nbinsx=30,
            hovertemplate="Range: %{x:,.0f}<br>Count: %{y}<extra></extra>"
        ),
        row=1, col=2
    )
    
    # 3. P-value distribution (excluding null values)
    p_values = df['p_value'].dropna()
    fig.add_trace(
        go.Histogram(
            x=p_values,
            name='P-Value Distribution',
            nbinsx=30,
            hovertemplate="P-Value: %{x}<br>Count: %{y}<extra></extra>"
        ),
        row=2, col=1
    )
    
    # 4. Scatter plot of registration volume vs p-value
    fig.add_trace(
        go.Scatter(
            x=df['lambda'],
            y=df['p_value'],
            mode='markers',
            name='Volume vs P-Value',
            text=df['zip'],  # Add ZIP codes for hover info
            marker=dict(
                size=8,
                color=df['total_evs'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="Total EVs")
            ),
            hovertemplate="ZIP: %{text}<br>Annual Avg: %{x:,.0f}<br>P-Value: %{y:.3f}<extra></extra>"
        ),
        row=2, col=2
    )
    
    # Update layout with better styling
    fig.update_layout(
        height=800,
        width=1200,
        showlegend=False,
        title_text="Vehicle Registration Analysis Dashboard",
        title_x=0.5,  # Center the title
        template="plotly_white",  # Use a clean template
        font=dict(family="Arial, sans-serif", size=12),
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="ZIP Code", row=1, col=1)
    fig.update_yaxes(title_text="Annual Average Registrations", row=1, col=1)
    fig.update_xaxes(title_text="Annual Average Registrations", row=1, col=2)
    fig.update_yaxes(title_text="Count", row=1, col=2)
    fig.update_xaxes(title_text="P-Value", row=2, col=1)
    fig.update_yaxes(title_text="Count", row=2, col=1)
    fig.update_xaxes(title_text="Annual Average Registrations", row=2, col=2)
    fig.update_yaxes(title_text="P-Value", row=2, col=2)
    
    # Add better hover interactions
    fig.update_layout(hovermode='closest')
    
    # Save as interactive HTML
    fig.write_html(
        output_path,
        include_plotlyjs=True,  
        full_html=True,  
        config={
            'displayModeBar': True,  
            'scrollZoom': True,      
            'modeBarButtonsToAdd': ['drawopenpath', 'eraseshape'],  
            'toImageButtonOptions': {'height': None, 'width': None}  
        }
    )
    
    print(f"Interactive dashboard has been saved to {output_path}")
    return fig


if __name__ == "__main__":
    try:

        fig = create_interactive_analysis('result_data/result.csv', 'Interactive_output/vehicle_analysis_dashboard.html')
        

        df = pd.read_csv('result_data/result.csv')
        print("\nData Summary:")
        print("-" * 50)
        print(f"Total ZIP codes analyzed: {len(df)}")
        print(f"Average vehicles per ZIP code: {df['lambda'].mean():,.0f}")
        print(f"Maximum vehicles in a ZIP code: {df['lambda'].max():,.0f}")
        print(f"Minimum vehicles in a ZIP code: {df['lambda'].min():,.0f}")
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")

Interactive dashboard has been saved to vehicle_analysis_dashboard.html

Data Summary:
--------------------------------------------------
Total ZIP codes analyzed: 109
Average vehicles per ZIP code: 498,524
Maximum vehicles in a ZIP code: 7,694,076
Minimum vehicles in a ZIP code: 27
